# Setup 

In [216]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from thefuzz import fuzz
from thefuzz import process

# MER

## Load data

In [217]:
MER = pd.read_excel('../data/MER.xlsx', sheet_name=[0, 1])
MERT, MERPPL = MER[0], MER[1]
MERT

,Invoicedate,Distributor,GroupId,Type,Provincename,MatNo,Material Name,New_Volume(L)
0,2023-09-06,A,ERW_AMBP2020110207,Reseller,Chachoengsao,145577,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,11676
1,2023-12-20,A,ERW_AMBP2020110207,Reseller,Chachoengsao,145577,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,5768
2,2023-08-11,A,ERW_AMBP2020110207,Reseller,Chachoengsao,143160,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,11088
3,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,143160,MOBIL DELVAC SUPER 1400 15W-40 4X6+1/CAR,2912
4,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,143160,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,7168
...,...,...,...,...,...,...,...,...
9424,2023-03-11,C,ERW_C08P2020090135,Retails,Lampang,140417,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",281
9425,2023-07-18,C,ERW_C08P2020090145,Retails,Chiang Mai,140417,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",354
9426,2023-10-19,C,ERW_C08P2020090203,Retails,Chiang Mai,140635,"MOBIL 1 FS X2 5W50, 12X1L CTN",281
9427,2023-11-29,C,ERW_C08P2020090203,Retails,Chiang Mai,140635,"MOBIL 1 FS X2 5W50, 12X1L CTN",162


### Column types

In [218]:
MERT.dtypes
MERPPL.dtypes

MatNo     int64
Points    int64
dtype: object

### Check null (general)

In [219]:
isNullMERT = MERT.isna().any().any()
isNullMERPPL = MERPPL.isna().any().any()
print(f"MERT: {str(isNullMERT)}, MERPPL: {str(isNullMERPPL)}")

MERT: False, MERPPL: False


### Check duplicated (general)

In [220]:
isDuplicatedMERT = MERT.duplicated().any()
isDuplicatedMERPPL = MERPPL.duplicated().any()

print(f"MERT: {str(isDuplicatedMERT)}, MERPPL: {str(isDuplicatedMERT)}")

MERT: False, MERPPL: False


## Investigate columns

### Invoicedate

In [221]:
dataType = str(MERT['Invoicedate'].dtypes)
invoicedateMin = MERT['Invoicedate'].min()
invoicedateMax = MERT['Invoicedate'].max()
print(f"Type: {dataType}, Min: {invoicedateMin}, Max:{invoicedateMax}")

Type: datetime64[ns], Min: 2023-01-03 00:00:00, Max:2023-12-29 00:00:00


The data is for the year 2023

### Distrubutor

In [222]:
MERT['Distributor'].unique()

array(['A', 'B', 'C', 'D'], dtype=object)

### GroupId

In [223]:
MERT['GroupId'].unique()
groupidMin = MERT['GroupId'].min()
groupidMax = MERT['GroupId'].max()
print(f"Min: {str(groupidMin)}, Max: {str(groupidMax)}")

Min: ERW_A1DS2023040002, Max: ERW_DYZP2023100001


### Type

In [224]:
MERT['Type'].unique()

array(['Reseller', 'Retails'], dtype=object)

### Provincename

In [225]:
MERT['Provincename'].unique() # found 'N\\A'
nullProvincename = MERT[MERT['Provincename'] == 'N\\A']['GroupId'].unique().tolist()
MERT[MERT['GroupId'].isin(nullProvincename)] # yep no way to fill this

,Invoicedate,Distributor,GroupId,Type,Provincename,MatNo,Material Name,New_Volume(L)
1059,2023-12-15,A,ERW_A7QF2022060003,Retails,N\A,145577,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,672
1771,2023-11-23,A,ERW_A7QF2022060003,Retails,N\A,145472,M DEL MODERN 10W-30 S DEFENS V2 4X7L/CAR,476
2489,2023-11-23,A,ERW_A7QF2022060003,Retails,N\A,145570,M SUP Friction Fighter 10W-40 4x4+1L/CAR,4320
3191,2023-03-17,A,ERW_A6CM2022070002,Retails,N\A,143629,MOBIL SUPER 2000 TBS 10W-30 4X7L/CAR,9492
3192,2023-07-21,A,ERW_A6CM2022070002,Retails,N\A,145577,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,9212
4427,2023-02-14,A,ERW_A7QF2022060003,Retails,N\A,144373,MOBIL DELVAC MX 10W-30 CTN 4X6+1L:TH,6692
4754,2023-12-15,A,ERW_A7QF2022060003,Retails,N\A,145570,M SUP Friction Fighter 10W-40 4x4+1L/CAR,3760
4954,2023-02-13,A,ERW_A7QF2022060003,Retails,N\A,142923,MOBIL SPECIAL 20W-50 CTN 4X4L:TH,6544
5690,2023-05-08,B,ERW_BXEO2021030004,Retails,N\A,143629,"โมบิล SUPER 2000 TBS 10W30 ,4/7",3164
5805,2023-11-03,A,ERW_A7QF2022060002,Retails,N\A,140043,MOBIL ATF 3309 12x1L/CAR,5392


Since A and B both share Bangkok and Bangkok is the highest market share, assume that all of these groupid are from Bangkok

In [226]:
MERT.loc[MERT['Provincename'] == 'N\A', 'Provincename'] = 'Bangkok'
MERT['Provincename'].unique()

array(['Chachoengsao', 'Nakhon Ratchasima', 'Nakhon Pathom', 'Phetchabun',
       'Rayong', 'Khon Kaen', 'Trat', 'Suphan Buri', 'Lop Buri',
       'Bangkok', 'Kanchanaburi', 'Chai Nat', 'Chon Buri', 'Phuket',
       'Udon Thani', 'Saraburi', 'Ratchaburi', 'Nakhon Si Thammarat',
       'Chainat', 'Chanthaburi', 'Chumphon', 'Yasothon', 'Chiang Rai',
       'Phangnga', 'Songkhla', 'Ubon Ratchathani', 'Maha Sarakham', 'Tak',
       'Chiang Mai', 'Samut Sakhon', 'Prachuap Khiri Khan', 'Krabi',
       'Pathum Thani', 'Roi Et', 'Sukhothai', 'Nakhon Sawan',
       'Surat Thani', 'Phra Nakhon Si Ayutthaya', 'Nong Bua Lam Phu',
       'Sing Buri', 'Nakhon Phanom', 'Nakhon Nayok', 'Surin', 'Uttaradit',
       'Lampang', 'Sa Kaeo', 'Lamphun', 'Pattani', 'Trang',
       'Kamphaeng Phet', 'Phitsanulok', 'Uthai Thani', 'Phrae', 'Phichit',
       'Sakon Nakhon', 'Loei', 'Nonthaburi', 'Samut Prakan', 'Nong Khai',
       'Phetchaburi', 'Nan', 'Buri Ram', 'Si Sa Ket', 'Prachin Buri',
       'Kalasin', 'P

### MatNo

In [227]:
MERT['MatNo'].unique()

array([145577, 143160, 145438, 145500, 145498, 145497, 145474, 141539,
       143877, 143629, 145575, 142939, 145570, 143507, 142930, 140918,
       142931, 143628, 135896, 145472, 135899, 145451, 144673, 135508,
       145444, 134728, 145888, 140521, 140043, 145826, 137227, 145841,
       145496, 145828, 140418, 145832, 134707, 143623, 124213, 143586,
       135898, 135895, 137232, 142938, 144373, 143566, 135509, 143898,
       135904, 143132, 145644, 140363, 141559, 145742, 146130, 144463,
       142923, 134346, 137219, 145569, 139840, 135903, 144473, 123150,
       143900, 145439, 143508, 135892, 145450, 143625, 134525, 143133,
       143110, 143899, 135872, 140417, 143874, 144672, 134349, 134345,
       145572, 145473, 143354, 145499, 143162, 134782, 145573, 135162,
       142924, 145447, 143106, 143574, 134533, 142212, 145930, 140709,
       133405, 124216, 142889, 145838, 145840, 140523, 144462, 145837,
       140635, 143576, 145727, 143099, 145449, 142932, 139706, 139759,
      

### Material Name

In [228]:
def extractName(series):    
    name = series.name
    
    # extract viscosity, None
    #pattern 1 _00w-00_
    viscosity = series.str.extract(r'\b(\d+)W-(\d+)\b') 
    #pattern 2 -00w00-
    viscosity.loc[(viscosity.isna().all(axis=1))] = series[viscosity.isna().all(axis=1)].str.extract(r'(\d{2})W(\d{2})')
    #pattern 3 _00w00_
    viscosity.loc[(viscosity.isna().all(axis=1))] = series[viscosity.isna().all(axis=1)].str.extract(r'\b(\d+)W(\d+)\b')
    #pattern 4 00w-00
    viscosity.loc[(viscosity.isna().all(axis=1))] = series[viscosity.isna().all(axis=1)].str.extract(r'(\d+)W-(\d+)')
    
    # if any pattern is found
    value = viscosity.dropna()
    if value.empty:
        finalVis = np.nan
    else:
        finalVis = (value[0].astype(str) + "w-" + value[1].astype(str)).values[0]


    # special pattern single w: 10w
    if finalVis is np.nan:
        specialVis = series.str.extract(r'\b(\d+)W\b')
        
        specialVisvalue = specialVis.dropna()

        if not specialVisvalue.empty:
            finalVis = str(specialVisvalue.values[0][0]) + "w"

    # isDelvac: True, or False

    #pattern: contain Del (case insensitive)
    containDel = series.str.contains('del', case=False).any()
    if containDel:
        finalDel = True
    else:
        finalDel = False


    # product line: super, one, modern, legend None
    containSuper = series.str.contains('sup', case=False).any()
    containOne = series.str.contains(r'\s1\s',regex=True).any()
    containModern = series.str.contains('modern', case=False).any()
    containLegend = series.str.contains('legend', case=False).any()

    if containLegend:
        finalLine = "Legend"
    elif containModern:
        finalLine = "Modern"
    elif containSuper:
        finalLine = "Super"
    elif containOne:
        finalLine = "One"
    else:
        finalLine = np.nan

    # product group name: just select one
    groupName = series.values[0]

    # Package
    finalPack = np.nan

    #pattern 1: _axb+1, regex: (\d)X(\d)\+1, (\d{1,2})X(\d{1,2})\+1, (\d{1,2})[xX](\d{1,2})\+1
    package = series.str.extract(r'(\d{1,2})[xX](\d{1,2})\+1')
    #pattern 2: a/b+1, regex: (\d)/(\d)\+1, (\d{1,2})/(\d{1,2})\+1
    package.loc[(package.isna().all(axis=1))] = series[package.isna().all(axis=1)].str.extract(r'(\d{1,2})/(\d{1,2})\+1')
    
    packageFound = package.dropna()
    if not packageFound.empty:
        finalPack = (packageFound[0].astype(str) + "x" + packageFound[1].astype(str) + "L+1").values[0]
        
    if finalPack is np.nan:
    #special pattern: M-BRAKE FLUID DOT4 12X0.5L
        package = series.str.extract(r'(\d+(?:\.\d+)?)X(\d+(?:\.\d+)?)[Ll]')
    #pattern 3: axb, regex: (\d)X(\d), (\d{1,2})X(\d{1,2}), (\d{1,2})[xX](\d{1,2}), (\d{1,2})[xX](\d{1,2})[Ll]
        package.loc[(package.isna().all(axis=1))] = series[package.isna().all(axis=1)].str.extract(r'(\d{1,2})[xX](\d{1,2})[Ll]')
    #pattern 4: a/b, regex: (\d)/(\d), (\d{1,2})/(\d{1,2})
        package.loc[(package.isna().all(axis=1))] = series[package.isna().all(axis=1)].str.extract(r'(\d{1,2})/(\d{1,2})')
        
        packageFound = package.dropna()
        if not packageFound.empty:
            finalPack = (packageFound[0].astype(str) + "x" + packageFound[1].astype(str) + "L").values[0]
    
    # One digit: Special case
    if finalPack is np.nan:
    #pattern 5: aL, regex: (\d)L, (\d+)L
        package = series.str.extract(r'(\d+)L')
        packageFound = package.dropna()
        if not packageFound.empty:
            finalPack = str(packageFound.values[0][0]) + "L"

    cols = ['Viscosity', 'isDelvac', 'ProductLine', 'Package','GroupName']
    
    return pd.Series((finalVis, finalDel, finalLine, finalPack, groupName), index=cols)
    




pd.set_option('display.max_colwidth', None)
MaterialTable = MERT.groupby('MatNo')['Material Name'].apply(extractName).unstack(-1)

In [229]:
MaterialTable

,Viscosity,isDelvac,ProductLine,Package,GroupName
MatNo,,,,,
123150,NaN,False,NaN,208L,"MOBIL MULTIPURPOSE ATF, 208LT DR"
124213,NaN,False,NaN,208L,MOBIL ATF 220 D 208LT
124216,NaN,False,NaN,1x20L,MOBIL ATF 220 P1/20LT
133405,20w-50,True,Super,12x1L,"MOBIL DELVAC SUPER 20W-50, 12X1LT CTN"
134345,10w,False,NaN,4x5L,"MOBIL HYDRAULIC 10W, 4X5LT CTN"
...,...,...,...,...,...
145840,0w-40,False,One,12x1L,MOBIL 1 0W-40 (SP) V2 12X1L/CAR
145841,0w-40,False,One,4x4L,MOBIL 1 0W-40 (SP) V2 4X4L/CAR
145888,20w-50,False,Super,4x4L+1,M SUP EVD Protection 20W-50 4x4+1L/CAR


In [230]:
MaterialTable["Package"].unique()

array(['208L', '1x20L', '12x1L', '4x5L', '1x18L', '12x0.5L', '4x6L',
       '20L', '4x4L', '1x35L', '18L', nan, '4x6L+1', '2x4L', '4x3.5L',
       '4x4L+1', '4x7L'], dtype=object)

In [236]:
MaterialTable[MaterialTable["Package"].isna()]

,Viscosity,isDelvac,ProductLine,Package,GroupName
MatNo,,,,,
141515,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG"


In [237]:
# join it back to MER
MERT = MERT.set_index("MatNo").join(MaterialTable)

In [238]:
MERT

,Invoicedate,Distributor,GroupId,Type,Provincename,Material Name,New_Volume(L),Viscosity,isDelvac,ProductLine,Package,GroupName
MatNo,,,,,,,,,,,,
145577,2023-09-06,A,ERW_AMBP2020110207,Reseller,Chachoengsao,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,11676,10w-30,False,Super,4x7L,M SUP Frict Fighter TBS 10W-30 4x7L/CAR
145577,2023-12-20,A,ERW_AMBP2020110207,Reseller,Chachoengsao,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,5768,10w-30,False,Super,4x7L,M SUP Frict Fighter TBS 10W-30 4x7L/CAR
143160,2023-08-11,A,ERW_AMBP2020110207,Reseller,Chachoengsao,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,11088,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR
143160,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,MOBIL DELVAC SUPER 1400 15W-40 4X6+1/CAR,2912,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR
143160,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,7168,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR
...,...,...,...,...,...,...,...,...,...,...,...,...
140417,2023-03-11,C,ERW_C08P2020090135,Retails,Lampang,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",281,5w-40,False,One,12x1L,MOBIL 1 TBD 5W-40 12X1L/CAR
140417,2023-07-18,C,ERW_C08P2020090145,Retails,Chiang Mai,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",354,5w-40,False,One,12x1L,MOBIL 1 TBD 5W-40 12X1L/CAR
140635,2023-10-19,C,ERW_C08P2020090203,Retails,Chiang Mai,"MOBIL 1 FS X2 5W50, 12X1L CTN",281,5w-50,False,One,12x1L,MOBIL 1 FS X2 5W-50 12x1L/CAR


### Volume

In [239]:
MERTVolumeMin = MERT['New_Volume(L)'].min()
MERTVolumeMax = MERT['New_Volume(L)'].max()
print(f"min: {MERTVolumeMin}, max: {MERTVolumeMax}") # seems alright!

min: 6, max: 14000


### Point (PPL)

In [240]:
MERPPL['Points'].unique()


array([10,  4,  2,  1])

## Join two table (Transcraction and Points)

In [241]:
MERT = MERT.join(MERPPL.set_index("MatNo"))
MERT

,Invoicedate,Distributor,GroupId,Type,Provincename,Material Name,New_Volume(L),Viscosity,isDelvac,ProductLine,Package,GroupName,Points
MatNo,,,,,,,,,,,,,
145577,2023-09-06,A,ERW_AMBP2020110207,Reseller,Chachoengsao,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,11676,10w-30,False,Super,4x7L,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,2
145577,2023-12-20,A,ERW_AMBP2020110207,Reseller,Chachoengsao,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,5768,10w-30,False,Super,4x7L,M SUP Frict Fighter TBS 10W-30 4x7L/CAR,2
143160,2023-08-11,A,ERW_AMBP2020110207,Reseller,Chachoengsao,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,11088,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,2
143160,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,MOBIL DELVAC SUPER 1400 15W-40 4X6+1/CAR,2912,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,2
143160,2023-03-10,A,ERW_AMBP2020110291,Reseller,Nakhon Ratchasima,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,7168,15w-40,True,Super,4x6L+1,MOBIL DELVAC SUPER1400 15W-40 4X6+1L/CAR,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140417,2023-03-11,C,ERW_C08P2020090135,Retails,Lampang,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",281,5w-40,False,One,12x1L,MOBIL 1 TBD 5W-40 12X1L/CAR,10
140417,2023-07-18,C,ERW_C08P2020090145,Retails,Chiang Mai,"MOBIL 1 TURBO DIESEL 5W-40, 12X1LT CTN",354,5w-40,False,One,12x1L,MOBIL 1 TBD 5W-40 12X1L/CAR,10
140635,2023-10-19,C,ERW_C08P2020090203,Retails,Chiang Mai,"MOBIL 1 FS X2 5W50, 12X1L CTN",281,5w-50,False,One,12x1L,MOBIL 1 FS X2 5W-50 12x1L/CAR,10


In [242]:
MERT['Points'].unique()

array([ 2,  1, 10,  4])

In [243]:
MERT['Points'].isna().any() # join successfully!

np.False_

In [244]:
MERT.reset_index().to_csv("../data/MERCleanedV2.csv")

In [245]:
MERT[MERT['Package'].isna()] # ignore this 

,Invoicedate,Distributor,GroupId,Type,Provincename,Material Name,New_Volume(L),Viscosity,isDelvac,ProductLine,Package,GroupName,Points
MatNo,,,,,,,,,,,,,
141515,2023-02-15,A,ERW_AMBP2020110283,Retails,Bangkok,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",280,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-02-13,A,ERW_AMBP2020110283,Retails,Bangkok,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",352,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-04-05,A,ERW_AMBP2020110681,Retails,Roi Et,M DEL EXTRALONGLIFE GREASE 6X1.8KG/CAR,366,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-02-02,C,ERW_C08P2020090174,Retails,Chiang Rai,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",207,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-01-06,C,ERW_C08P2020090212,Retails,Nakhon Sawan,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",82,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-01-06,C,ERW_C08P2020090021,Retails,Kamphaeng Phet,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",435,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-07-25,D,ERW_DBG72020110082,Reseller,Songkhla,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",96,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
141515,2023-08-11,D,ERW_DBG72020110082,Reseller,Songkhla,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",23,NaN,True,NaN,NaN,"M-DEL EXTRA LONG LIFE GREASE, 1.8KGX6:SG",4
